In [1]:
import jn_setup
#from simulation_procedure import model, mode
from tools.paths import *
from simulation_procedure import make_model
import pandas as pd, pyabc, hydroeval as he, numpy as np, uuid

To activate debug mode, run 'inp_path = set_inp_path('debug')'.
To activate test mode, run 'inp_path = set_inp_path('test')'.
To activate run mode, run 'inp_path = set_inp_path('run')'.


In [2]:
mode = "test"
model = make_model(mode = mode, swmm_cleanup = 'none', vvwm_cleanup = 'none')

C:\Users\jstelman\Git\stelman_urban_pesticides\master_test\NPlesantCreek.inp


In [3]:
"""
Priors. Get the values from that csv. Just for SWMM at first.
"""
swmm_ranges = pd.read_csv(os.path.join(master_path, "lhs_param_ranges.csv"), index_col=0,
                           usecols = ["Parameter","Min", "Range"])

'''
Link up with the vvwm priors and make one big list with 36 params
'''
vvwm_ranges = pd.read_csv(os.path.join(master_path, "lhs_param_ranges_vvwm.csv"), index_col=0,
                           usecols = ["Parameter","Min", "Range"])

param_ranges = pd.concat([swmm_ranges, vvwm_ranges], axis = 0)

if mode == "debug":
    param_ranges = param_ranges.loc[['NImperv','kd']]

priors = param_ranges.to_dict("index")

# borrowed from Jeff: <https://github.com/JeffreyMinucci/bee_neonic_abc/blob/master/pyabc_run.ipynb>
prior = pyabc.Distribution(**{key: pyabc.RV("uniform", loc = v['Min'], scale = v['Range'])
                        for key, v in priors.items()})

In [4]:
priors

{'NImperv': {'Min': 0.01, 'Range': 0.015},
 'NPerv': {'Min': 0.05, 'Range': 0.45},
 'SImperv': {'Min': 1.27, 'Range': 1.27},
 'SPerv': {'Min': 2.54, 'Range': 2.54},
 'PctZero': {'Min': 0.01, 'Range': 99.99},
 'MaxRate': {'Min': 8.46, 'Range': 118.54},
 'MinRate': {'Min': 0.254, 'Range': 10.666},
 'Decay': {'Min': 2.0, 'Range': 5.0},
 'DryTime': {'Min': 2.0, 'Range': 12.0},
 'Por': {'Min': 0.4, 'Range': 0.1},
 'FC': {'Min': 0.06, 'Range': 0.32},
 'WP': {'Min': 0.024, 'Range': 0.241},
 'Ksat': {'Min': 0.25, 'Range': 12.45},
 'Rough': {'Min': 0.01, 'Range': 0.016},
 'Kdecay': {'Min': 0.002739726, 'Range': 0.197260274},
 'BCoeff2': {'Min': 0.5, 'Range': 1.5},
 'WCoeff2': {'Min': 0.066, 'Range': 0.148},
 'kd': {'Min': 882.0, 'Range': 5028.0},
 'aer_aq': {'Min': 5.0, 'Range': 360.0},
 'aer_aq_temp': {'Min': 20.0, 'Range': 5.0},
 'anae_aq': {'Min': 5.0, 'Range': 725.0},
 'anae_aq_temp': {'Min': 20.0, 'Range': 5.0},
 'photo': {'Min': 96.9, 'Range': 319.1},
 'rflat': {'Min': 40.0, 'Range': 0.0}

In [5]:
prior

<Distribution 'BCoeff2', 'Decay', 'DryTime', 'FC', 'Kdecay', 'Ksat', 'MaxRate', 'MinRate', 'NImperv', 'NPerv', 'PctZero', 'Por', 'Rough', 'SImperv', 'SPerv', 'WCoeff2', 'WP', 'aer_aq', 'aer_aq_temp', 'anae_aq', 'anae_aq_temp', 'benthic_depth', 'bnmas', 'bulk_density', 'chl', 'doc1', 'doc2', 'froc1', 'froc2', 'hydro', 'kd', 'photo', 'porosity', 'rflat', 'sol', 'sused'>

# Make the .new object
### 1. Import observed data

In [6]:
# import it again to make inspect it
# specifically for TEST mode!
if mode == 'debug':
    with open(os.path.join(main_path, 'master_debug','debug_obs_data.txt'),'r') as read_file:
        obs_dict = eval(read_file.read())
elif mode == 'test':
    with open(os.path.join(main_path, 'master_test','test_obs_data.txt'),'r') as read_file:
        obs_dict = eval(read_file.read())
elif mode == 'run':
    with open(os.path.join(main_path, 'master','obs_data.txt'),'r') as read_file:
        obs_dict = eval(read_file.read())
obs_dict

{'2009-02-13_26': 0.017,
 '2009-02-13_28': 0.0485,
 '2009-02-13_29': 0.203,
 '2009-04-07_28': 0.0192,
 '2009-04-07_29': 0.0985,
 '2009-04-08_26': 0.00774,
 '2009-04-13_26': 0.00088,
 '2009-04-13_28': 0.00858,
 '2009-04-13_29': 0.0257,
 '2009-05-01_26': 0.0088,
 '2009-05-01_28': 0.0145,
 '2009-05-01_29': 0.0203,
 '2009-08-28_26': 0.00088,
 '2009-08-28_28': 0.00553,
 '2009-08-28_29': 0.0168,
 '2009-10-12_26': 0.00088,
 '2009-10-12_28': 0.0074,
 '2009-10-12_29': 0.0141,
 '2009-10-13_26': 0.0195,
 '2009-10-13_28': 0.0513,
 '2009-10-13_29': 0.0332,
 '2010-04-04_26': 0.0114,
 '2010-04-04_28': 0.0388,
 '2010-04-04_29': 0.0405,
 '2010-07-19_26': 0.00088,
 '2010-07-19_28': 0.0146,
 '2010-07-19_29': 0.0343,
 '2010-07-19_42': 0.00088,
 '2010-10-24_26': 0.00088,
 '2010-10-24_29': 0.0317,
 '2010-10-24_35': 0.0094,
 '2010-10-24_38': 0.0067,
 '2010-10-24_42': 0.00088,
 '2011-02-16_26': 0.0799,
 '2011-02-16_29': 0.064,
 '2011-02-16_35': 0.0251,
 '2011-02-16_38': 0.0368,
 '2011-02-16_42': 0.00088}

### 2. Initialize dask client for dask distributed sampler

In [7]:
# from dask.distributed import Client#, LocalCluster
# cluster = LocalCluster()#n_workers=(90/2), threads_per_worker = 2)  # Set for 96 vCPU compute instance
# client = Client(cluster)#,timeout=400)

# make it simpler
# if __name__ == "__main__":
# client = Client()

# sampler = pyabc.sampler.DaskDistributedSampler(dask_client = client)

# See if this takes all those errors out
sampler = pyabc.sampler.SingleCoreSampler()
# make the process more transparent
sampler.sample_factory.record_rejected = True
sampler.show_progress = True

### 3. Set up a sqlite db directory

In [8]:
# Initialize a new ABC inference run
dbid = uuid.uuid4().hex[0:8]
print(dbid)
database_dir = os.path.join(temp_path, 'results_db')  
if not os.path.exists(database_dir):
    os.mkdir(database_dir)
db_path = ("sqlite:///" +
           os.path.join(database_dir, "test_pyabc_" + dbid + ".db"))

6edebc99


### 4. Defining a Distance function

We need to refactor the NSE distance function using the pyabc.Distance class.
We will need the hydroeval library and the pyabc.SimpleFunctionDistance to do this

In [9]:
# make a file to hold onto these NSEs for our own record
with open(os.path.join(temp_path, "NSEs_" + dbid + ".txt"), "w") as nse_file:
    nse_file.write("NSEs\n")

In [10]:
def nse(x, x_0):
    nse = he.evaluator(he.nse, 
                       simulation_s = np.array(list(x.values())), 
                       evaluation = np.array(list(x_0.values())))[0]
    print("nse ", nse)
    # make record
    with open(os.path.join(temp_path, "NSEs_" + dbid + ".txt"),"a") as nse_file:
        nse_file.write(str(nse)+"\n")
    return nse
    
NSE = pyabc.SimpleFunctionDistance(fun=nse)

# the best answer is 1
# make one that measures distance from 1
NSED = pyabc.SimpleFunctionDistance(fun = lambda x, x_0: 1 - nse(x, x_0))

### 5. Define ABCSMC object

In [11]:
abc = pyabc.ABCSMC(model, prior, 
                   # might fix the dask problem too
                   population_size = pyabc.ConstantPopulationSize(40), # just to shorten the run
                   sampler = sampler,
                   distance_function = NSED)

In [12]:
abc

### 6. Initialize a new abc run

In [13]:
abc.new(db_path, obs_dict)

In [14]:
# Back to 1 gen
history = abc.run(max_nr_populations=1, minimum_epsilon=0.2)

Folder  3bebe181  created 

Folder  3bebe181 outfall_31_26  created 

Folder  3bebe181 outfall_31_28  created 

Folder  3bebe181 outfall_31_29  created 

Folder  3bebe181 outfall_31_35  created 

Folder  3bebe181 outfall_31_36  created 

Folder  3bebe181 outfall_31_38  created 

Folder  3bebe181 outfall_31_42  created 

 25% |██████                  | 1/4 Folder  d6c99df9  created 

Folder  d6c99df9 outfall_31_26  created 

Folder  d6c99df9 outfall_31_28  created 

Folder  d6c99df9 outfall_31_29  created 

Folder  d6c99df9 outfall_31_35  created 

Folder  d6c99df9 outfall_31_36  created 

Folder  d6c99df9 outfall_31_38  created 

Folder  d6c99df9 outfall_31_42  created 

 50% |████████████            | 2/4 Folder  10987ec3  created 

Folder  10987ec3 outfall_31_26  created 

Folder  10987ec3 outfall_31_28  created 

Folder  10987ec3 outfall_31_29  created 

Folder  10987ec3 outfall_31_35  created 

Folder  10987ec3 outfall_31_36  created 

Folder  10987ec3 outfall_31_38  created 

Fold

C:\Users\jstelman\Anaconda3\envs\swmm1\lib\site-packages\scipy\stats\_distn_infrastructure.py:1835: RuntimeWarning: invalid value encountered in double_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)


Folder  1e4f8d13 outfall_31_26  created 

Folder  1e4f8d13 outfall_31_28  created 

Folder  1e4f8d13 outfall_31_29  created 

Folder  1e4f8d13 outfall_31_35  created 

Folder  1e4f8d13 outfall_31_36  created 

Folder  1e4f8d13 outfall_31_38  created 

Folder  1e4f8d13 outfall_31_42  created 

nse  -7.31400231419859e+23
 50% |████████████            | 2/4 Folder  d40dc443  created 

Folder  d40dc443 outfall_31_26  created 

Folder  d40dc443 outfall_31_28  created 

Folder  d40dc443 outfall_31_29  created 

Folder  d40dc443 outfall_31_35  created 

Folder  d40dc443 outfall_31_36  created 

Folder  d40dc443 outfall_31_38  created 

Folder  d40dc443 outfall_31_42  created 

nse  -7.31400231419859e+23
 75% |██████████████████      | 3/4 Folder  27d930d0  created 

Folder  27d930d0 outfall_31_26  created 

Folder  27d930d0 outfall_31_28  created 

Folder  27d930d0 outfall_31_29  created 

Folder  27d930d0 outfall_31_35  created 

Folder  27d930d0 outfall_31_36  created 

Folder  27d930d0 out

TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'